# QBM Comparison Exact vs. Annealer

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import neptune.new as neptune
from numba import njit
from scipy.constants import k as k_B, h as h_P

k_B /= h_P * 1e9

from qbm.models import BQRBM
from qbm.plotting import plot_qq
from qbm.utils import (
    binarize_df,
    convert_bin_list_to_str,
    get_binarization_params,
    get_project_dir,
    get_rng,
    kl_divergence,
    load_artifact,
    lr_exp_decay,
    prepare_training_data,
    save_artifact,
    unbinarize_df,
)
from qbm.utils.exact_qbm import get_pauli_kron, compute_H, compute_rho

# configure directories
project_dir = get_project_dir()
artifact_dir = project_dir / "artifacts/qbm/8x4"
if not artifact_dir.exists():
    artifact_dir.mkdir(parents=True)
plot_dir = project_dir / "results/plots/qbm/8x4"
if not plot_dir.exists():
    plot_dir.mkdir(parents=True)
    
# load anneal schedule
df_anneal = pd.read_csv(
    project_dir
    / "data/anneal_schedules/csv/09-1265A-A_Advantage_system5_1_annealing_schedule.csv",
    index_col="s",
)
if 0.5 not in df_anneal.index:
    df_anneal.loc[0.5] = (df_anneal.loc[0.499] + df_anneal.loc[0.501]) / 2

time: 1.84 s (started: 2022-01-29 21:07:22 +01:00)


## Train Data Creation

In [2]:
seed = 42
n_visible = 8
n_hidden = 4
n_qubits = n_visible + n_hidden

rng = get_rng(seed)
n_samples = 1500
α = 2 / 3
N_1 = rng.normal(-2, 1, int(round(n_samples * α, 0)))
N_2 = rng.normal(3, 1, int(round(n_samples * (1 - α), 0)))
x = np.concatenate((N_1, N_2))

df = pd.DataFrame.from_dict({"x": x})
binarization_params = get_binarization_params(df, n_bits=n_visible)
df_binarized = binarize_df(df, binarization_params)
X_train = prepare_training_data(df_binarized)["X_train"]

time: 11.2 ms (started: 2022-01-29 21:07:24 +01:00)


In [3]:
def callback(model, sample_state_vectors):
    X_train = model.X_train
    n_visible = model.X_train.shape[1]
    train_states = (
        model._eigen_to_binary(model.X_train) * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)
    sample_states = (
        model._eigen_to_binary(sample_state_vectors[:, :n_visible])
        * 2.0 ** np.arange(n_visible - 1, -1, -1)
    ).sum(axis=1)

    dkl = kl_divergence(train_states, sample_states, n_bins=32)

    return {"value": dkl, "print": f"D_KL = {dkl:.3f}"}

time: 1.14 ms (started: 2022-01-29 21:07:24 +01:00)


## Model Analysis (Exact)

In [4]:
# model params
s = 1.0
embedding = None
beta_initial = 0.5
exact_params = {"beta": 0.5}

# training params
n_epochs = 100
n_samples = 10_000
learning_rate = 0.1
decay_epoch = 50
decay_period = 10
epochs = np.arange(1, n_epochs + 1)
learning_rates = learning_rate * lr_exp_decay(epochs, decay_epoch=50, period=10)
learning_rates_beta = learning_rate * lr_exp_decay(epochs, decay_epoch=50, period=20)

anneal_params = {
    "s": s,
    "A": df_anneal.loc[s, "A(s) (GHz)"],
    "B": df_anneal.loc[s, "B(s) (GHz)"],
}

train_models = True
models_dir = artifact_dir / f"models/mini_batch_comparison"
mini_batch_sizes = [10, 16, 32, 64, 100, 128]
for mini_batch_size in mini_batch_sizes:
    if train_models:
        # model init
        model_exact = BQRBM(
            X_train=X_train,
            n_hidden=n_hidden,
            embedding=embedding,
            anneal_params=anneal_params,
            beta_initial=beta_initial,
            exact_params=exact_params,
        )

        # model train and save
        model_exact.train(
            n_epochs=n_epochs,
            n_samples=n_samples,
            learning_rate=learning_rates,
            learning_rate_beta=learning_rates_beta,
            mini_batch_size=mini_batch_size,
            callback=callback,
        )
        model_exact.save(models_dir / f"model_exact-s={s}-mbs={mini_batch_size}.pkl")
        model_exact_metrics = {
            "A": model_exact.A,
            "B": model_exact.B,
            "a": model_exact.a,
            "b": model_exact.b,
            "W": model_exact.W,
            "beta": model_exact.beta,
            "embedding": model_exact.embedding,
            "anneal_params": model_exact.anneal_params,
            "exact_params": model_exact.exact_params,
            "beta_history": model_exact.beta_history,
            "callback_outputs": [x for x in model_exact.callback_outputs],
        }
        save_artifact(
            model_exact_metrics,
            models_dir / f"model_exact-s={s}-mbs={mini_batch_size}-attributes.pkl",
        )

model_paths = [x for x in models_dir.iterdir() if "attributes" not in str(x)]
models = {}
for model_path in model_paths:
    mbs = int([x for x in model_path.stem.split("-") if "mbs" in x][0].split("=")[1])
    models[mbs] = BQRBM.load(model_path)

[BQRBM] epoch 1: β = 0.481, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.618640
	D_KL = 0.190
[BQRBM] epoch 2: β = 0.468, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.596006
	D_KL = 0.092
[BQRBM] epoch 3: β = 0.462, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.580453
	D_KL = 0.065
[BQRBM] epoch 4: β = 0.464, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.610706
	D_KL = 0.047
[BQRBM] epoch 5: β = 0.468, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.578613
	D_KL = 0.054
[BQRBM] epoch 6: β = 0.473, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.597837
	D_KL = 0.055
[BQRBM] epoch 7: β = 0.477, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:02.585692
	D_KL = 0.046
[BQRBM] epoch 8: β = 0.480, learning rate = 1.00e-01, β learning rate = 1.00e-01, epoch duration = 0:00:

In [ ]:
markers = ["o", "^", "v", "<", ">", "s", "p", "*", "P", "X"]
colors = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan",
]

fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
for i, mbs in enumerate(mini_batch_sizes):
    dkls = [x["value"] for x in models[mbs].callback_outputs]
    epochs = np.arange(1, len(dkls) + 1)
    ax.plot(epochs, dkls, label=f"Batch Size = {mbs}", color=colors[i], marker=markers[i], markevery=3)
    
ax.set_xlabel("Epoch")
ax.set_ylabel(r"$D_{KL}(p_{data} \ || \ p_{model})$")
ax.set_ylim((0, 0.5))
ax.set_yticks(np.arange(0, 55, 5) / 100)
ax.grid()
ax.legend()
plt.savefig(plot_dir / "batch_size_comparison.png")